In [98]:
import cv2
from skimage.filters import threshold_otsu
import numpy as np
from skimage.morphology import (erosion, dilation, opening, closing,  # noqa
                                white_tophat)
from skimage.measure import find_contours


In [189]:
img= cv2.imread("er2.png") 
gray_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
t =threshold_otsu(gray_img)
binarized_img=np.where(gray_img>t,255,0)
#detct corners
#detect some corners inside of words
#does not detect lines connected to attributes and some times relations
#(img,max number of corners,quality of corner,min distance between corners)
corners = cv2.goodFeaturesToTrack(gray_img, 50, 0.0001, 20)
corners = np.int0(corners)
for i in corners:
    x, y = i.ravel()
    cv2.circle(img, (x, y), 3, 255, -1)
cv2.imwrite('corners.png', img)

True

In [192]:
img= cv2.imread("er2.png") 
im = cv2.imread("er2.png", cv2.IMREAD_GRAYSCALE)
th, im_th = cv2.threshold(im_in, 220, 255, cv2.THRESH_BINARY_INV)
corners = cv2.goodFeaturesToTrack(im, 50, 0.0001, 30)
corners = np.int0(corners)
# Floodfill from corners
# cv2.circle(img, (x1, y1), 3, (255, 0, 0) , -1)
# cv2.floodFill(im_th, mask, (x1,y1), 255)
# cv2.imwrite('first_corner.png', img)
# for i in corners:
#     x, y = i.ravel()
#     cv2.floodFill(im_th, mask, (x,y), 255)


In [193]:
#harris corner detection
gray_float = np.float32(gray_img)
dst = cv2.cornerHarris(gray_float,4,3,0.05)

#result is dilated for marking the corners, not important
#dst = cv2.dilate(dst,None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.02*dst.max()]=[0,0,255]
cv2.imwrite('harris.png', img)

True

In [214]:
im_in = cv2.imread("er2.png", cv2.IMREAD_GRAYSCALE)
th, im_th = cv2.threshold(im_in, 220, 255, cv2.THRESH_BINARY_INV)

im_floodfill = im_th.copy()
# Mask used to flood filling.
# Notice the size needs to be 2 pixels than the image.
h, w = im_th.shape[:2]
mask = np.zeros((h+2, w+2), np.uint8)
# Floodfill from point (0, 0)
cv2.floodFill(im_floodfill, mask, (0,0), 255)
cv2.floodFill(im_floodfill, mask, (w-1,0), 255)
cv2.floodFill(im_floodfill, mask, (0,h-1), 255)
cv2.floodFill(im_floodfill, mask, (w-1,h-1), 255)

# Invert floodfilled image
im_floodfill_inv = cv2.bitwise_not(im_floodfill)
# Combine the two images to get the foreground.
im_out = im_th | im_floodfill_inv
# Display images.
cv2.imwrite("Thresholded_Image.png", im_th)
cv2.imwrite("Floodfilled_Image.png", im_floodfill)
cv2.imwrite("Inverted_Floodfilled_Image.png", im_floodfill_inv)
cv2.imwrite("Foreground.png", im_out)
#start flooding from corners?

True

In [196]:
#mophological close
structureElm =  np.array((
	[1,1,1,1],
	[1,1,1,1],
    [1,1,1,1]), dtype="int")
closed = closing(im_th, structureElm)

for i in range(50):
    closed = closing(closed, structureElm)
cv2.imwrite("closed.png", closed)

#mophological opening
structureElm =  np.array((
	[1,1,1],
	[1,1,1],
    [1,1,1]), dtype="int")
opened = dilation(im_th, structureElm)
for i in range(10):
    opened = opening(opened, structureElm)
cv2.imwrite("opened.png", opened)



True

In [197]:
im = cv2.imread("er2.png", cv2.IMREAD_GRAYSCALE)
kernel1 = np.array((
    [0,0,0],
	[-1,1,-1],
    [0,0,0]), dtype="int")
kernel2 = np.array((
	[0,-1,0],
    [0,1,0],
    [0,-1,0]), dtype="int")
kernel3 = np.array((
	[-1,0,0],
    [0,1,0],
    [0,0,-1]), dtype="int")
kernel4 = np.array((
	[0,0,-1],
    [0,1,0],
    [-1,0,0]), dtype="int")
kernel5 = np.array((
	[0,-1,0],
    [-1,4,-1],
    [0,-1,0]), dtype="int")
#only bar edges after floodfilling the image
image_with_edges = cv2.filter2D(im_floodfill, -1, kernel1)
image_with_edges += cv2.filter2D(im_floodfill, -1, kernel2)
image_with_edges += cv2.filter2D(im_floodfill, -1, kernel3)
image_with_edges += cv2.filter2D(im_floodfill, -1,kernel4)
#by subtraction we get all step edges
edges = cv2.filter2D(im_floodfill, -1, kernel5)

edges_ored = cv2.bitwise_not(image_with_edges & im_floodfill)
cv2.imwrite("edges.png", image_with_edges)
cv2.imwrite("edges2.png", edges)
cv2.imwrite("edges_ored.png", edges_ored)




True

In [184]:
img= cv2.imread("er2.png") 
im = cv2.imread("er2.png", cv2.IMREAD_GRAYSCALE)
th, im_th = cv2.threshold(im_in, 220, 255, cv2.THRESH_BINARY_INV)
corners = cv2.goodFeaturesToTrack(im, 50, 0.0001, 30)
corners = np.int0(corners)

# h, w = im_th.shape[:2]
# mask = np.zeros((h+2, w+2), np.uint8)
# # Floodfill from corners
# x1,y1 = corners[1].ravel()
# cv2.circle(img, (x1, y1), 3, (255, 0, 0) , -1)
# cv2.imwrite('first_corner.png', img)

# cv2.floodFill(im_th,seedPoint= (x1,y1),newVal=(0,255,0),mask=mask)
for i in corners:
    x, y = i.ravel()
    cv2.circle(img, (x, y), 2, (255, 0, 0) , -1)
#     cv2.floodFill(im_th, mask, (x,y), 255)
# cv2.imwrite('flooded_corners.png', im_th)
cv2.imwrite('corners.png', img)
closed = 1-closed

i=0
area = closed.shape[0]*closed.shape[1]
contours = find_contours(opened,0.1)
for contour in contours:
    x = contour[:,1]
    y = contour[:,0]
    [Xmin, Xmax, Ymin, Ymax] = [np.amin(x), np.amax(x), np.amin(y), np.amax(y)]
    c = im[int(Ymin):int(Ymax),int(Xmin):int(Xmax)]
    cv2.imwrite("contours/"+str(i)+".png",c)
    i+=1


In [ ]:

# Display the image and plot all contours found
##################Plot contours################
plt.imshow(im_floodfill, interpolation='nearest')

for n, contour in enumerate(contours):
    plt.plot(contour[:, 1], contour[:, 0], linewidth=2)

plt.axis('image')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
def floodFillUtil(imgFlood, xImg, yImg):
    #h,w
    N,M = imgFlood.shape[:2]
    if (xImg < 0 or xImg >= M or yImg < 0 or yImg >= N):
        return
    if (imgFlood[yImg][xImg] == 0):
        return
    # Replace the color at (x, y)
    imgFlood[yImg][xImg] = 255
    # Recur for north, east, south and west
    floodFillUtil(imgFlood, xImg+1, yImg )
    floodFillUtil(imgFlood, xImg-1, yImg )
    floodFillUtil(imgFlood, xImg  , yImg+1 )
    floodFillUtil(imgFlood, xImg  , yImg-1 )
 
def floodFillCustom(imgFlood, xImg, yImg):
    prevC = imgFlood[yImg][xImg]
    if(prevC==0):
        return
    floodFillUtil(imgFlood, xImg, yImg)